In [1]:
import os
import time
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.3.1


In [79]:
# Total categories
label_dim = 10

In [80]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [81]:
training_size = len(train_images)
test_size = len(test_images)

In [82]:
# Scaling the train and test images on 0-1 scale
train_images = np.array(train_images) / 255
test_images = np.array(test_images) / 255

In [83]:
# Reshaping to required N * H * W * C 
train_images = train_images.reshape(training_size, 28, 28, 1)
test_images = test_images.reshape(test_size, 28, 28, 1)

In [84]:
# Applying one hot encoding to the images
train_labels = tf.keras.utils.to_categorical(train_labels, label_dim)
test_labels = tf.keras.utils.to_categorical(test_labels, label_dim)

In [85]:
# Casting the labels to float -> As they are scaled bet 0-1
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [86]:
print(train_labels.shape)
print(test_labels.shape)

(60000, 10)
(10000, 10)


In [87]:
# Building a model now
inputs = tf.keras.layers.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
pred = tf.keras.layers.Dense(label_dim, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=pred)
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 576)              

In [88]:
# compiling the model
optimizer = tf.keras.optimizers.SGD()
model.compile(loss='categorical_crossentropy',
             optimizer=optimizer,
             metrics=['accuracy'])

In [89]:
# We will run on CPU. Convert model to estimator for easy use
strategy = None
config = tf.estimator.RunConfig(train_distribute=strategy)
estimator = tf.keras.estimator.model_to_estimator(model, config=config)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/z_/rwjk4yhx7w71xlx5qwtmfdj40000gp/T/tmpipl5sp0n', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [90]:
# Define training and testing functions
def input_fn(images, labels, epochs, batch_size):
    # Convert inputs to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    
    # Shuffle, repeat, and batch the examples
    shuffle_size = 5000
    dataset = dataset.shuffle(shuffle_size).repeat(epochs).batch(batch_size)
    dataset = dataset.prefetch(None)
    
    # return dataset
    return dataset
    
    

In [91]:
# Training
batch_size = 512
epochs = 50
estimator_train_result = estimator.train(input_fn=lambda:input_fn(train_images, train_labels,
                                                          epochs=epochs, batch_size=batch_size))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/var/folders/z_/rwjk4yhx7w71xlx5qwtmfdj40000gp/T/tmpipl5sp0n/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: /var/folders/z_/rwjk4yhx7w71xlx5qwtmfdj40000gp/T/tmpipl5sp0n/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 10 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/z_/rwjk4yhx7w71xlx5qwtmfdj40000gp/T/tmpipl5sp0n/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0..

In [92]:
print(estimator_train_result)

In [93]:
estimator.evaluate(lambda:input_fn(test_images, test_labels, epochs=1, batch_size=batch_size))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-12-29T19:28:16Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/z_/rwjk4yhx7w71xlx5qwtmfdj40000gp/T/tmpipl5sp0n/model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 3.08386s
INFO:tensorflow:Finished evaluation at 2020-12-29-19:28:19
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.8202, global_step = 5860, loss = 0.48910326
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /var/folders/z_/rwjk4yhx7w71xlx5qwtmfdj40000gp/T/tmpipl5sp0n/model.ckpt-5860


{'accuracy': 0.8202, 'loss': 0.48910326, 'global_step': 5860}